# Spark context 초기화

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col

sqlContext = SQLContext(sc)
bucket = "fc-class"     

# Text File Read

In [ ]:
log = sc.textFile("s3://"+ bucket +"/ods/danji_master.csv").map(lambda line:line.split(","))

# CSV 파일 읽기

In [ ]:
csvDF= spark.read.format("csv").option("header","true").load("s3://"+ bucket +"/ods/danji_master.csv")

# CSV 파일 읽기 
데이터 확인

In [ ]:
csvDF.show()

# CSV To Json 쓰기

In [ ]:
csvDF.coalesce(1).write.mode("overwrite").format('json').save("s3://"+ bucket+"/pyspark/class/danji_master.json")


# Json File Read
Source 읽기

In [ ]:
df_json = spark.read.json("s3://"+ bucket+"/pyspark/class/danji_master.json")

# Json File Read
데이터 확인

In [ ]:
df_json.show()

# Json To Parquet File Write
Parquet 로 쓴다

In [ ]:
df_json.coalesce(1).write.mode("overwrite"). \
format('parquet').save("s3://"+ bucket+"/pyspark/class/danji_master.parquet")

# Parquet File Read
Create Temp View Table

In [ ]:
df_pq = spark.read.parquet("s3://"+ bucket+"/pyspark/class/danji_master.parquet")

In [ ]:
df_pq.createOrReplaceTempView("danji")

# Table 확인

In [ ]:
spark.sql("show tables").show()

# Table 조회 하기

In [ ]:
spark.sql("select count(*) from danji").show()

In [ ]:
%%sql

select * from danji

# DB 읽기

실습 6주차 참고하기


In [ ]:
jdbcHostname = "pipeline-instance-1.cp7ovqbewwso.ap-northeast-2.rds.amazonaws.com"
jdbcDatabase = "datamart"
username="admin"
password="epdlxj!"
jdbcPort = 3306
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}?characterEncoding=UTF-8".format(jdbcHostname, jdbcPort, jdbcDatabase, username, password)

sql="(select * from datamart.danji_master_data15 limit 10) a"
jdbcDF = spark.read.format("jdbc").option("driver","com.mysql.jdbc.Driver" ).option("url", jdbcUrl).option("dbtable", sql).option("user", username).option("password", password).load()

jdbcDF.show()

In [ ]:
sqlContext.cacheTable("danji")

In [ ]:
%%sql

select dong, count(*)
from danji
group by dong
order by count(*) desc

In [ ]:

# jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}?characterEncoding=UTF-8".format(jdbcHostname, jdbcPort, jdbcDatabase, username, password)

In [ ]:
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {   
  "user" : username,
  "password" : password,
  "driver" : "com.mysql.jdbc.Driver"
}

In [ ]:
sql="(select * from datamart.danji_master_data15 limit 10) a"
jdbcDF = spark.read.format("jdbc").option("driver","com.mysql.jdbc.Driver" ).option("url", jdbcUrl).option("dbtable", sql).option("user", username).option("password", password).load()


In [ ]:
jdbcDF.show()